<a href="https://colab.research.google.com/github/Alexjmsherman/nlp_practicum_cohort3_instructor/blob/master/lessons/lesson_8_text_similarity/text_similarity_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Similarity

##### Author: Alex Sherman | alsherman@deloitte.com, Vaibhav Srivastav (vasrivastav@deloitte.com) | Revannth (revedala@deloitte.com)

### Agenda:

- Levenshtein distance
- Jaccard distance
- Euclidean distance
- Cosine similarity
- Similarity with [sklearn, gensim, spacy]
- Similarity with word embeddings (Word2vec similarity, Word movers distance)
- Similarity of Probability Distributions (Cross entropy, KL Divergence, Hellinger distance)
- Nearest-neighbors search
- Clustering

"we often want to determine **similarity between pairs of documents**, or the **similarity between a specific document** and a set of other documents (such as a user query vs. indexed documents).


### Use cases:
Broad:
- information retrieval
- document clustering
- word-sense disambiguation
- automatic essay scoring
- short answer grading
- machine translation
- Recommendation Engines
- Search Engines (Query - Result matching)

Specific:
- Grant similarity
- Complaint similarity
- Duplicate questions
- Question Answering (e.g. give the same style of question to a customer support agent)

In [1]:
from configparser import ConfigParser, ExtendedInterpolation
from collections import defaultdict
from itertools import combinations, product
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim import corpora, models, similarities
from gensim.corpora import Dictionary
from gensim.matutils import kullback_leibler, hellinger
from gensim.models import ldamodel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity
from gensim.summarization.bm25 import get_bm25_weights
from gensim.utils import simple_preprocess
from IPython.core.display import display, HTML
from IPython.display import Image
from IPython.lib.display import YouTubeVideo
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy.matcher import Matcher
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
import spacy

# increase display of columns in pandas
pd.set_option('display.max_colwidth', 200)

In [2]:
CLEANED_TEXT_PATH ='../input/data-input/data_immunity.csv'
SIMILARITY_INDEX = 'pubmed.index'

In [3]:
# configuration for data, acronyms, and gensim paths
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

GENSIM_DICTIONARY_PATH = config['NLP']['GENSIM_DICTIONARY_PATH']
GENSIM_CORPUS_PATH = config['NLP']['GENSIM_CORPUS_PATH']
CLEANED_TEXT_PATH = config['NLP']['CLEANED_TEXT_PATH']
SIMILARITY_INDEX = config['NLP']['SIMILARITY_INDEX']
SAVED_EMBEDDINGS_PATH = config['EMBEDDINGS']['SAVED_EMBEDDINGS_PATH']
REPORTER_ZIP_PATH = config['EMBEDDINGS']['REPORTER_ZIP_PATH']

KeyError: 'NLP'

### Review Cleaned Text

In [4]:
cleaned_text_df = pd.read_csv(CLEANED_TEXT_PATH, sep='\t',header=None,names=['text'])
cleaned_text = cleaned_text_df.text.values
cleaned_text_df.head()

FileNotFoundError: [Errno 2] File b'../input/data-input/data_immunity.csv' does not exist: b'../input/data-input/data_immunity.csv'

### Text Similarity Approaches

In [5]:
# # text similarity approaches
# # SOURCE: https://pdfs.semanticscholar.org/5b5c/a878c534aee3882a038ef9e82f46e102131b.pdf
# Image("https://s3.amazonaws.com/nlp.practicum/text_similarity_approaches.png", width=350)

![](https://s3.amazonaws.com/nlp.practicum/text_similarity_approaches.png)

## Character Based Text Similarity

"As an example, this technology is used by information retrieval systems, search engines, automatic indexing systems, text summarizers, categorization systems, plagiarism checkers, speech recognition, rating systems, DNA analysis, and profiling algorithms (IR/AI programs to automatically link data between people and what they do)."

##### The Levenshtein Distance

"This method was invented in 1965 by the Russian Mathematician Vladimir Levenshtein (1935-2017). The distance value describes the minimal number of deletions, insertions, or substitutions that are required to transform one string (the source) into another (the target). Unlike the Hamming distance, the Levenshtein distance works on strings with an unequal length."

SOURCE: [Levenshtein Distance and Text Similarity in Python](http://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/)

In [6]:
seq1 = 'medicine'
seq2 = 'medical'

# create a matrix
size_x = len(seq1) + 1
size_y = len(seq2) + 1
matrix = np.zeros ((size_x, size_y))
matrix

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
# set col numbers (0, n-1)
for x in range(size_x):
    matrix [x, 0] = x

# set row numbers (0, n-1)
for y in range(size_y):
    matrix [0, y] = y

matrix

array([[0., 1., 2., 3., 4., 5., 6., 7.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 0.],
       [6., 0., 0., 0., 0., 0., 0., 0.],
       [7., 0., 0., 0., 0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
import numpy as np

def levenshtein(seq1, seq2):
    # create a matrix
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    
    # set col numbers (0, n-1)
    for x in range(size_x):
        matrix [x, 0] = x
    
    # set row numbers (0, n-1)
    for y in range(size_y):
        matrix [0, y] = y

    # calculate distance
    for x in range(1, size_x):
        for y in range(1, size_y):
            # if characters match do not increase distance
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = matrix[x-1, y-1]
            # if characters don't match increase min distance by 1
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    # print the distance calulation matrix
    # list(seq1) converts string into a list of the characters
    print(pd.DataFrame(matrix[1:,1:], index=list(seq1), columns=list(seq2)))
    
    return (matrix[size_x - 1, size_y - 1])

levenshtein('medicine','medical')

     m    e    d    i    c    a    l
m  0.0  1.0  2.0  3.0  4.0  5.0  6.0
e  1.0  0.0  1.0  2.0  3.0  4.0  5.0
d  2.0  1.0  0.0  1.0  2.0  3.0  4.0
i  3.0  2.0  1.0  0.0  1.0  2.0  3.0
c  4.0  3.0  2.0  1.0  0.0  1.0  2.0
i  5.0  4.0  3.0  2.0  1.0  1.0  2.0
n  6.0  5.0  4.0  3.0  2.0  2.0  2.0
e  7.0  6.0  5.0  4.0  3.0  3.0  3.0


3.0

In [9]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo('We3YDTzNXEk')

## Term Based Text Similarity

In [10]:
# create a document-term matrix for the first 10 documents
vect = CountVectorizer(max_features=10)
transform_vect = vect.fit_transform(cleaned_text_df['text'][0:10])

# rename the indices as doc_#
index = ['doc_{}'.format(i) for i in range(10)]

# create a dataframe
dist_df = pd.DataFrame(
      transform_vect.todense()
    , columns=vect.get_feature_names()
    , index=index
)

# view two docs
dist_df.loc[['doc_1','doc_7'],:]

NameError: name 'cleaned_text_df' is not defined

### Jaccard Index

"The Jaccard Index is a statistic to compare and measure how similar two different sets to each other. It is a ratio of intersection of two sets over union of them.

If you have representative finite number of elements for a particular observation and you want to compare this observation with another observation, you could count the number of items that are common to both of these two sets. It is a natural fit for comparing posts if you know the representative tags for the posts to measure how similar two articles are in terms of tags."

SOURCE: http://bugra.github.io/work/notes/2017-02-07/similarity-via-jaccard-index/

In [11]:
# # Similarity via the Jaccard Index
# Image("https://s3.amazonaws.com/nlp.practicum/jaccard_index.png", width=450)

![](https://s3.amazonaws.com/nlp.practicum/jaccard_index.png)

In [12]:
# calculate the distance among different vectors
#
# NOTE: if both counts are zero, then the term in not counted in the denominator
#       as the term is outside of the union of both term sets

from sklearn.metrics.pairwise import pairwise_distances

# calculate jaccard distance on all documents
jaccard = pairwise_distances(dist_df.values, metric='jaccard')

NameError: name 'dist_df' is not defined

In [13]:
sns.heatmap(pairwise_distances(dist_df.values, Y=None, metric='jaccard'));

NameError: name 'dist_df' is not defined

In [14]:
for doc_ind in range(0,5):
    print('jaccard distance {}'.format(jaccard[0][doc_ind])) # focus on the first doc (index 0)
    print(dist_df.iloc[[0, doc_ind], :], '\n\n')  # print the first doc and the comparison document

NameError: name 'jaccard' is not defined

### Euclidean Distance

In [15]:
# the geometric equation of the dot product
# Image("https://s3.amazonaws.com/nlp.practicum/euclidean_distance.jpg", width=300)

![](https://s3.amazonaws.com/nlp.practicum/euclidean_distance.jpg)

In [16]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [17]:
# 3-4-5 triangle
p1 = np.array([0,0])
p2 = np.array([3,4])
euclidean_distance(x=p1, y=p2)

5.0

In [18]:
# 0-3, 0-4
p1 - p2

array([-3, -4])

In [19]:
# -3^2, -4^2
(p1 - p2) ** 2

array([ 9, 16])

In [20]:
# 9 + 16
np.sum((p1 - p2) ** 2)

25

In [21]:
# sqrt(25)
np.sqrt(np.sum((p1 - p2) ** 2))

5.0

In [22]:
# example from above diagram
p1 = np.array([5,11])
p2 = np.array([12,14])
euclidean_distance(x=p1, y=p2)

7.615773105863909

In [23]:
# euclidean distance scales beyond 2-D
p1 = np.array([0,0,0])
p2 = np.array([3,3,3])
euclidean_distance(x=p1, y=p2)

5.196152422706632

In [24]:
# the distance between a vect and itself is 0
euclid_df = dist_df 
euclidean_distance(euclid_df.iloc[0], euclid_df.iloc[0])

NameError: name 'dist_df' is not defined

In [25]:
# calculate the distance among different vectors
for doc_ind in range(1,8):
    print(euclidean_distance(euclid_df.iloc[0], euclid_df.iloc[doc_ind]))

NameError: name 'euclid_df' is not defined

In [26]:
print(pairwise_distances(euclid_df, Y=None, metric='euclidean')[0])
sns.heatmap(pairwise_distances(euclid_df, Y=None, metric='euclidean'));

NameError: name 'euclid_df' is not defined

### Cosine Similarity

##### Dot Product

"The dot product for two vectors of a and b where a_n and b_n are the components of the vector (features of the document or TF-IDF values for each word of the document in our example)"

Imagine two documents:
- a = 'medicine medicine medicine'
- b = 'biomedical biomedical biomedical biomedical'

In [27]:
# example dot product calculation
# Image("https://s3.amazonaws.com/nlp.practicum/dot_product_calculation.png", width=200)

![](https://s3.amazonaws.com/nlp.practicum/dot_product_calculation.png)

The CountVectorizer output of the two documents show the word count, where the vectors do not overlap for any words. Thus the dot product is zero.

In [28]:
# dot product equation
# Image("https://s3.amazonaws.com/nlp.practicum/dot_product.png", width=500)

![](https://s3.amazonaws.com/nlp.practicum/dot_product.png)

### The Norm

In [29]:
# the geometric equation of the dot product
# Image("https://s3.amazonaws.com/nlp.practicum/normalization.png", width=800)

![](https://s3.amazonaws.com/nlp.practicum/normalization.png)

In [30]:
# the geometric equation of the dot product
# Image("https://s3.amazonaws.com/nlp.practicum/normalization_example.png", width=800)

![](https://s3.amazonaws.com/nlp.practicum/normalization_example.png)

In [31]:
norm_url = 'https://rorasa.wordpress.com/2012/05/13/l0-norm-l1-norm-l2-norm-l-infinity-norm/'
iframe = '<iframe src={} width=1100 height=300></iframe>'.format(norm_url)
HTML(iframe)

##### cosine similarity
The cosine similarity between two vectors (or two documents on the Vector Space) is a measure that calculates the cosine of the angle between them. This metric is a measurement of orientation and not magnitude, it can be seen as a comparison between documents on a normalized space because we’re not only taking into the consideration the magnitude of each word count (tf-idf) of each document, but also the angle between the documents. What we have to do to build the cosine similarity equation is to solve the equation of the dot product for cosine:


That is the cosine similarity formula. Cosine Similarity will generate a metric that says how related are two documents by looking at the angle instead of the magnitude.

In [32]:
# the geometric equation of the dot product
# Image("https://s3.amazonaws.com/nlp.practicum/geometric_dot_product.png", width=200)

![](https://s3.amazonaws.com/nlp.practicum/geometric_dot_product.png)

In [33]:
# cosine similarity equation
# Image("https://s3.amazonaws.com/nlp.practicum/cosine_similarity.png", width=200)

![](https://s3.amazonaws.com/nlp.practicum/cosine_similarity.png)

In [34]:
# cosine similarity angles
# 1 = perfect match
# 0 = dissimilar documents
# Image("https://s3.amazonaws.com/nlp.practicum/cosine_similarity_angles.png", width=1200)

![](https://s3.amazonaws.com/nlp.practicum/cosine_similarity_angles.png)

In [35]:
cosine_url = 'http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/'
iframe = '<iframe src={} width=950 height=300></iframe>'.format(cosine_url)
HTML(iframe)

# Similarity with sklearn

### tfidf -> lsi -> cosine similarity (sklearn)

In [36]:
# view the tfidf scores
from sklearn.feature_extraction.text import TfidfVectorizer

# create tfidf matrix
df = pd.DataFrame(cleaned_text[0:5], columns=['text'])
vect = TfidfVectorizer(max_features=15)
transform_vect = vect.fit_transform(df['text'])
pd.DataFrame(transform_vect.toarray(), columns=vect.get_feature_names())

NameError: name 'cleaned_text' is not defined

In [37]:
# reduce the dimensions using SVD
from sklearn.decomposition import TruncatedSVD

# reduce dimensions of tfidf matrix
svd = TruncatedSVD(n_components=2)
svd_tranform = svd.fit_transform(transform_vect)
print(svd_tranform)

NameError: name 'transform_vect' is not defined

##### Build a recommendation engine with cosine similarity

In [38]:
from sklearn.neighbors import NearestNeighbors

In [39]:
# EXAMPLE

# sample data
X = [[1,1], [1,2], [1,3], [1,4], [1,5]]

# calculate cosine distance
nn = NearestNeighbors(metric='cosine')
nn.fit(X)

# build a graph of the nearest neighbors using cosine distance
nn_graph = nn.kneighbors_graph(n_neighbors=4, mode='distance').toarray()
pd.DataFrame(nn_graph)

,0,1,2,3,4
0,0.000000,0.051317,0.105573,0.142507,0.167950
1,0.051317,0.000000,0.010051,0.023813,0.035236
2,0.105573,0.010051,0.000000,0.002946,0.007722
3,0.142507,0.023813,0.002946,0.000000,0.001132
4,0.167950,0.035236,0.007722,0.001132,0.000000


In [40]:
# calculate cosine distance
cosine_neighbors = NearestNeighbors(metric='cosine')
cosine_neighbors.fit(svd_tranform)

NameError: name 'svd_tranform' is not defined

In [41]:
# build a graph of the nearest neighbors using cosine distance
ind = ['doc{}'.format(i) for i in range(1,6)]
graph = cosine_neighbors.kneighbors_graph(n_neighbors=2, mode='distance').toarray()
pd.DataFrame(graph, columns=ind, index=ind)

NotFittedError: This NearestNeighbors instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

The similarity in vector space models is determined by using associative coefficients based on the inner product of the document vector and query vector, where word overlap indicates similarity. The inner product is usually normalized. The most popular similarity measure is the cosine coefficient, which measures the angle between a document vector and the query vector.

Think about it this way. In the numerator of cosine similarity, only terms that exist in both documents contribute to the dot product. If both of the term have high tfidf values, then they add a lot to the numerator. If a term does not exist in either documents, then it adds nothing to the numerator. On the other hand, the deonominator normalizes the documents, so that a document with many terms is punished with a larger denominator. 


SOURCE: http://cogsys.imm.dtu.dk/thor/projects/multimedia/textmining/node5.html

### Problems with Cosine Distance
Cosine Distance does not require exact matches. Unfortunately, the bag-of-words approach implies a word order independence that can overstate document similarity. The sentences ***“I am not a crook”** versus **“Am I not a crook?”** would receive a perfect cosine similarity score of 1, despite their intentions being entirely different

### Additional Material - Weighted Cosine Similarity

Weighted cosine similarity measure: iteratively computes the cosine distance between two documents, but at each iteration the vocabulary is defined by n-grams of different lengths. 

The weighted similarity measure gives a single similarity score, but is built from the cosine similarity between two documents taken at several levels of coarseness. Put simply, we tokenize the two documents with unigrams, compute the cosine similarity between them, and then retokenize the documents with bigrams and again compute the similarity. We repeat this process until we reach our user-defined maximum n-gram length. The resulting set of scores is then combined as a weighted average, where heavier weights are given to the similarity scores between word frequency vectors from the higher n-gram lengths. 

Like the basic cosine similarity score, the weighted version varies from 0 to 1. the weighted cosine approach can capture pairs of documents where one copies but edits another. Since this is often the case in copied legislation, judicial decisions or regulations, we expect the weighted cosine similarity score to potentially better represent the similarity between technical documents


SOURCE:
- [Comparing and Evaluating Cosine Similarity Scores,
Weighted Cosine Similarity Scores, & Substring
Matching](http://hansen.web.unc.edu/files/2014/12/AHJS_Weighted_Cosine.pdf)

In [42]:
# weighted cosine similarity
# Image("https://s3.amazonaws.com/nlp.practicum/weighted_cosine_similarity.png", width=400)

![](https://s3.amazonaws.com/nlp.practicum/weighted_cosine_similarity.png)

##  Similarity with gensim (generate similarity)

##### bow -> tfidf -> lsi in gensim 

In [43]:
cleaned_text

NameError: name 'cleaned_text' is not defined

In [44]:
# convert sentences into list of tokens to format data for gensim
texts = [text.split() for text in cleaned_text]
print(texts[0])

NameError: name 'cleaned_text' is not defined

In [45]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

# filter the dictionary to remove infrequent (no_below) and too frequent (no_above) terminology
dictionary.filter_extremes(no_below=.01, no_above=0.99)

print(dictionary)

NameError: name 'texts' is not defined

In [46]:
# create bag of words (e.g. countvectorizer)
bow_corpus = [dictionary.doc2bow(sent) for sent in texts]

# create tfidf model (e.g. tfidf.fit )
tfidf = models.TfidfModel(bow_corpus)

# convert bow to tfidf (e.g. tfidf.transform)
tfidf_corpus = tfidf[bow_corpus] 

# view the first transformed sentence
print(tfidf_corpus[0])

NameError: name 'texts' is not defined

In [47]:
# create an lsi model (e.g. truncatedsvd.fit)
# corpus = data to fit model
# id2word = corpus stores words with numeric ids, this dict maps ids back to the original terms
lsi = models.LsiModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=20)
print(lsi)

NameError: name 'tfidf_corpus' is not defined

In [48]:
# convert the tfidf corpus to lsi (e.g. truncatedsvd.transform)
lsi_corpus = lsi[tfidf_corpus]
print(lsi_corpus[0])

NameError: name 'lsi' is not defined

### Index Documents for similarity recommendations (gensim)

**Compute the cosine similarity of a dynamic query against a static corpus of documents**

- gensim.similarities.**MatrixSimilarity**: for an efficient, memory-mapped index -- dense NumPy implementation


- gensim.similarities.**SparseMatrixSimilarity**: for an efficient, memory-mapped index -- sparse SciPy implementation


- gensim.similarities.**Similarity**: for an efficient out-of-core sharded index (auto-selects MatrixSimilarity or SparseMatrixSimilarity for each shard internally, based on the shard density); this is the most flexible class and should be your first choice.

In [49]:
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity, Similarity

# transform corpus to LSI space and index it
# output_prefix - local file to write index (required to enable later additions to the index)
# corpus - data to fit model
# num_features - often the num_terms in the corpus

index = Similarity(
      output_prefix='lsi.mm'
    , corpus=lsi_corpus
    , num_features=lsi.num_terms
    , shardsize=32768
)

NameError: name 'lsi_corpus' is not defined

In [50]:
index.save(SIMILARITY_INDEX)
!ls

NameError: name 'index' is not defined

In [51]:
index = Similarity.load(SIMILARITY_INDEX)
print(index)

FileNotFoundError: [Errno 2] No such file or directory: 'pubmed.index'

### Similarity Query

In [52]:
# tokenize a query string
query = "stimulator of beta_adrenergic structure in treatment_of_ischemic heart_disease".split()
print(query, '\n')

# convert to bag of words
vec_bow = dictionary.doc2bow(query)

# convert bow to tfidf
tfidf_bow = tfidf[vec_bow] 

# convert tfidf to LSI space to prepare for similarity query
vec_lsi = lsi[tfidf_bow]

print(vec_lsi)

['stimulator', 'of', 'beta_adrenergic', 'structure', 'in', 'treatment_of_ischemic', 'heart_disease'] 



NameError: name 'dictionary' is not defined

In [53]:
# set the number of matches to return (sorted by most relevant)
index.num_best = 10

# perform a similarity query against the index (similarity index of training docs)
sims = index[vec_lsi]

print('num docs: {}'.format(len(sims)))
sims

NameError: name 'index' is not defined

In [54]:
# NOTE: we add the same documents to the index twice, so the returned sims number may be out of range 
# for the original texts (if so just divide the index by two before slicing texts)
print(texts[(2291)])

NameError: name 'texts' is not defined

In [55]:
# add documents to a existing Similarity index (e.g. useful for big data and streaming)

print(index)  # before
index.add_documents(lsi_corpus)
print(index)  # after

NameError: name 'index' is not defined

##### NOTE: These queries only consider semantic similarity, and ignore important behavior driven features

### Annoy

##### Why use Annoy?

The current implementation for finding k nearest neighbors in a vector space in gensim has linear complexity via brute force in the number of indexed documents, although with extremely low constant factors. The retrieved results are exact, which is an overkill in many applications: approximate results retrieved in sub-linear time may be enough. Annoy can find approximate nearest neighbors much faster.

SOURCE: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/annoytutorial.ipynb

### BM25 (Best Match)

- [bm25 gensim source code](https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/summarization/bm25.py)

In [56]:
from gensim.summarization.bm25 import get_bm25_weights

In [57]:
%%time

scores = get_bm25_weights(bow_corpus[0:10])
print(scores[0])

NameError: name 'bow_corpus' is not defined

In [58]:
bm25_df = pd.DataFrame(scores).T
bm25_df['text'] = cleaned_text[0:10]
bm25_df.head()

NameError: name 'scores' is not defined

## Similarity with SpaCy

### Similarity

spaCy is able to compare two objects, and make a prediction of how similar they are. Predicting similarity is useful for building recommendation systems or flagging duplicates. For example, you can suggest a user content that's similar to what they're currently looking at, or label a support ticket as a duplicate if it's very similar to an already existing one.

In [59]:
!python -m spacy download en_core_web_lg

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/util/connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/conda/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionp

In [60]:
import en_core_web_lg
nlp = en_core_web_lg.load()

##### Document Similarity

In [61]:
from itertools import combinations

doc1 = nlp(u"Paris is the largest city in France.")
doc2 = nlp(u"Vilnius is the capital of Lithuania.")
doc3 = nlp(u"An emu is a large bird.")
doc4 = nlp(u"seagulls flying by the beach")

doc_combos = list(combinations([doc1, doc2, doc3, doc4], r=2))
doc_combos

[(Paris is the largest city in France., Vilnius is the capital of Lithuania.),
 (Paris is the largest city in France., An emu is a large bird.),
 (Paris is the largest city in France., seagulls flying by the beach),
 (Vilnius is the capital of Lithuania., An emu is a large bird.),
 (Vilnius is the capital of Lithuania., seagulls flying by the beach),
 (An emu is a large bird., seagulls flying by the beach)]

In [62]:
for doc, other_doc in doc_combos:
    print(doc)
    print(other_doc)
    print(doc.similarity(other_doc), '\n')

Paris is the largest city in France.
Vilnius is the capital of Lithuania.
0.7554965262954281 

Paris is the largest city in France.
An emu is a large bird.
0.6921461821156969 

Paris is the largest city in France.
seagulls flying by the beach
0.5691887753380548 

Vilnius is the capital of Lithuania.
An emu is a large bird.
0.5668025741640493 

Vilnius is the capital of Lithuania.
seagulls flying by the beach
0.4577827526596423 

An emu is a large bird.
seagulls flying by the beach
0.5974336979105327 



### Word2Vec Similarity Examples:

SOURCE: https://quomodocumque.wordpress.com/2016/01/15/messing-around-with-word2vec/

### Similarities in context

Aside from spaCy's built-in word vectors, which were trained on a lot of text with a wide vocabulary, the parsing, tagging and NER models also rely on vector representations of the meanings of words in context. As the processing pipeline is applied spaCy encodes a document's internal meaning representations as an array of floats, also called a tensor. This allows spaCy to make a reasonable guess at a word's meaning, based on its surrounding words. Even if a word hasn't been seen before, spaCy will know something about it. Because spaCy uses a 4-layer convolutional network, the tensors are sensitive to up to four words on either side of a word.

In [63]:
# Cartesian product of input iterables.  Equivalent to nested for-loops.
from itertools import product

In [64]:
list(product([['sentence1'],['sentence2'],['sentence3']], repeat=2))

[(['sentence1'], ['sentence1']),
 (['sentence1'], ['sentence2']),
 (['sentence1'], ['sentence3']),
 (['sentence2'], ['sentence1']),
 (['sentence2'], ['sentence2']),
 (['sentence2'], ['sentence3']),
 (['sentence3'], ['sentence1']),
 (['sentence3'], ['sentence2']),
 (['sentence3'], ['sentence3'])]

In [65]:
def create_similarity_matrix(texts, size, labels=None, viz=True):
    """ build a heatmap of similarity scores """

    # create a heatmap of similarities by iterating through each term 
    # and calculating its similarity to every other term
    # similarity is determined by comparing word vectors or "word embeddings"
    matrix = [t1.similarity(t2) for t1, t2 in product(texts, repeat=2)]
    
    # remove bottom half of matrix (i.e. duplicate data)
    matrix = np.array(matrix).reshape(size,size)    
    matrix = np.triu(matrix)
    
    # transform similarity scores into a dataframe
    if labels is None:
        labels = [term for term in texts]
    df = pd.DataFrame(matrix, columns=labels, index=labels)
    
    # create a heat map of the similarity scores
    if viz:
        sns.set(rc={'figure.figsize':(10,8)})
        sns.heatmap(df.values)
    
    return df

In [66]:
#sns.set(rc={'figure.figsize':(12,10)})
create_similarity_matrix(
      nlp(cleaned_text[0])
    , size=len(cleaned_text[0].split())
)

NameError: name 'cleaned_text' is not defined

##### Sentence Similarity using the full dataset

In [67]:
from itertools import combinations

In [68]:
list(combinations([['sentence1'],['sentence2'],['sentence3']], r=2))

[(['sentence1'], ['sentence2']),
 (['sentence1'], ['sentence3']),
 (['sentence2'], ['sentence3'])]

In [69]:
%%time

# use set to avoid looking at the similarity of dupliate sentences
docs = [nlp(sentence) for sentence in set(cleaned_text[0:1000])]

NameError: name 'cleaned_text' is not defined

In [70]:
%%time

# compare the similarity of all sentences against each other
similarity_scores = []
for doc1, doc2 in combinations(docs, r=2):
    sentences_are_different = doc1.text != doc2.text
    if sentences_are_different:
        similarity_scores.append([doc1.text, doc2.text, doc1.similarity(doc2)])

# view the top most similar sentences
# drop sentences with greater than .99 similarity as these are often duplicates
df = pd.DataFrame(similarity_scores, columns=['doc1','doc2','similarity'])
df[df.similarity < .97].sort_values('similarity', ascending=False).head(10)

NameError: name 'docs' is not defined

### Word Movers Distance

INSTALLATION: conda install -c conda-forge pyemd 

From Word Embeddings To Document Distances Abstract: http://proceedings.mlr.press/v37/kusnerb15.pdf

In [71]:
import gensim.downloader as api
word_vectors = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors from gensim-data

ValueError: unable to read local cache '/tmp/gensim-data/information.json' during fallback, connect to the Internet and retry

In [72]:
# Word Mover's Distance

# example sentences
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

# Compute WMD.
distance = word_vectors.wv.wmdistance(document1=sentence_obama, document2=sentence_president)
distance

NameError: name 'word_vectors' is not defined

In [73]:
doc1.text

'Paris is the largest city in France.'

In [74]:
%%time

# compare the similarity of all sentences against each other
similarity_scores = []
for doc1, doc2 in combinations(docs, r=2):
    sentences_are_different = doc1.text != doc2.text
    if sentences_are_different:
        wmd = word_vectors.wv.wmdistance(
            document1=simple_preprocess(doc1.text)
          , document2=simple_preprocess(doc2.text)
        )
        similarity_scores.append([doc1.text, doc2.text, wmd])

NameError: name 'docs' is not defined

In [75]:
df = pd.DataFrame(similarity_scores, columns=['doc1','doc2','word_movers_distance'])
df = df[df.word_movers_distance != np.inf]
df.sort_values('word_movers_distance', ascending=True).head(10)

,doc1,doc2,word_movers_distance


# Similarity of Probability Distributions

In [76]:
from gensim.utils import simple_preprocess
from gensim.matutils import kullback_leibler, hellinger

In [77]:
# view the first document
cleaned_text_df['text'][1]

NameError: name 'cleaned_text_df' is not defined

In [78]:
# use gensim to preprocess the document
print(simple_preprocess(cleaned_text_df['text'][1]))

NameError: name 'cleaned_text_df' is not defined

In [79]:
# preprocess all documents
clean_text = [simple_preprocess(text) for text in cleaned_text_df['text']]

# build bag of words
dictionary = Dictionary(clean_text)
corpus = [dictionary.doc2bow(text) for text in clean_text]

# create LDA model
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

NameError: name 'cleaned_text_df' is not defined

In [80]:
# create a bag of words for select documents
doc1 = model.id2word.doc2bow(simple_preprocess('abdominal pain persisted throughout treatment'))
doc2 = model.id2word.doc2bow(simple_preprocess('the patient had stomach discomfort'))
doc3 = model.id2word.doc2bow(simple_preprocess('the president greets the press in chicago'))

# we can now get the LDA topic distributions for these documents
doc1_dist = model[doc1] 
doc2_dist = model[doc2] 
doc3_dist = model[doc3] 

# view results
print('doc1_dist: {}'.format(doc1_dist))
print('doc2_dist: {}'.format(doc2_dist))
print('doc3_dist: {}'.format(doc3_dist))

NameError: name 'model' is not defined

##### Kullback Leibler Divergence

Kullback-Leibler returns a value in range [0, +∞) where values closer to 0 mean less distance (higher similarity).

The KL divergence is a measure of how much “predictive power” or “evidence” each sample will on average bring when you’re trying to distinguish p(x) from q(x), if you’re sampling from p(x). If p(x) and q(x) are very similar then each individual sample will bring little “evidence” to the table. On the other hand, if p(x) and q(x) are very different then each sample will bring a lot of evidence showcasing that q(x) is not a good representation of p(x).

In practice you’re often in situations where you want to build a model that’s as close as possible to the “true” model. In that case, you would like it to be as difficult as possible to distinguish the model you built from the real one, especially for samples that have been sampled from the real model.

SOURCE: https://medium.com/@cotra.marko/making-sense-of-the-kullback-leibler-kl-divergence-b0d57ee10e0a

##### Great Explanation of Entropy, Cross-Entropy, and KL-Divergence

[A Short Introduction to Entropy, Cross-Entropy and KL-Divergence](https://www.youtube.com/watch?v=ErfnhcEV1O8)

In [81]:
kullback_leibler(doc1_dist, doc2_dist)

NameError: name 'doc1_dist' is not defined

In [82]:
kullback_leibler(doc1_dist, doc3_dist)

NameError: name 'doc1_dist' is not defined

##### Hellinger distance

The Hellinger distance metric gives an output in the range [0,1] for two probability distributions, with values closer to 0 meaning they are more similar.

SOURCE: https://en.wikipedia.org/wiki/Hellinger_distance

In [83]:
hellinger(doc1_dist, doc2_dist)

NameError: name 'doc1_dist' is not defined

In [84]:
hellinger(doc1_dist, doc3_dist)

NameError: name 'doc1_dist' is not defined

# Sentence Clustering

In [85]:
# file names for lesson
NIH_EXPORTER_CSV = r'https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2019_042.zip'
cols = ['PROJECT_TITLE']
reporter = pd.read_csv(NIH_EXPORTER_CSV, encoding='latin-1', usecols=cols)

# remove duplicates
reporter = reporter[~reporter.PROJECT_TITLE.duplicated()]

reporter.shape

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [86]:
# vectorize the text
vect = TfidfVectorizer(min_df=.01, max_df=.95, norm='l2', stop_words='english', max_features=1000, ngram_range=(1,2))
titles_vect = vect.fit_transform(reporter['PROJECT_TITLE'])

NameError: name 'reporter' is not defined

In [87]:
%%time

# cluster the document using KMeans

# step 1 - import the model
from sklearn.cluster import KMeans

# step 2 - instantiate the model
km = KMeans(n_clusters=100, random_state=42)

# step 3 - fit the model with data
# clustering is unsupervised so we do not have labels to add during .fit()
km.fit(titles_vect)

# step 4 - predict the cluster of each section_title
reporter['clusters'] = km.predict(titles_vect)

NameError: name 'titles_vect' is not defined

In [88]:
def review_clusters(df, n_clusters):
    for cl_num in range(n_clusters):
        print(cl_num)
        print(reporter[df.clusters == cl_num]['PROJECT_TITLE'].values[0:10])
        print()

review_clusters(reporter, n_clusters=100)

NameError: name 'reporter' is not defined